<a href="https://colab.research.google.com/github/dviva1972/denvlaiva/blob/master/dll_hw__5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DLL

## Домашняя работа 5  |  RNN
## Иванов Денис

## Задание 1

1. Напишите алгоритм шифра цезаря для генерации выборки 

2.  Создайте и обучите нейронную сеть (вход - зашифрованная фраза, выход - дешифрованная фраза)

3. Проверьте качество модели

In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import time

import random

1.Алгоритм шифра Цезаря 

In [2]:
def Сaesar(string, num):
    output = ''
    for c in string:
        if c.isalpha():
            new_num = ord(c) + num
            if new_num > ord('z'):
                new_num -= 26
            output += chr(new_num)
        else:
            output += c
    return output

In [3]:
Сaesar('trew', 2)

'vtgy'

Сгенерим данные на базе изречений симпсонов

In [4]:
import os
import re
import sys
from google.colab import drive 

In [5]:
import copy

In [6]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_csv('/content/drive/My Drive/DLL/simpsons/data.csv', encoding='utf-8-sig').iloc[:,[-2]]
df = df.dropna(subset=['normalized_text'])

df['shift']   = [random.randint(1, 10) for i in range(len(df))]
df['text_in'] = [' '.join(re.findall('[\w]+', i)) for i in df['normalized_text']]
df['text_out']= df.loc[:, ['text_in', 'shift']].apply(
                        lambda row: Сaesar(row['text_in'], row['shift']), axis=1)

df = df.iloc[:,1:]
df.head()

,shift,text_in,text_out
0,5,maggie look whats that,rfllnj qttp bmfyx ymfy
1,3,lee mur lee mur,ohh pxu ohh pxu
2,3,zee boo zee boo,chh err chh err
3,2,im trying to teach maggie that nature doesnt e...,ko vtakpi vq vgcej ociikg vjcv pcvwtg fqgupv g...
4,8,its like an ox only it has a hump and a dewlap...,qba tqsm iv wf wvtg qb pia i pcux ivl i lmetix...


In [10]:
[[c for c in ph] for ph in df.text_in if type(ph) is not str]

[]

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train, test = train_test_split(df, test_size=0.2)
train_text  = train['text_out' ].tolist()
train_label = train['text_in'].tolist()
test_text   = test[ 'text_out' ].tolist()
test_label  = test[ 'text_in'].tolist()

In [13]:
INDEX_TO_CHAR = ['none'] + [w for w in set('abcdefghijklmnopqrstuvwxyz ')]

dict_in  = {w: i for i, w in enumerate(INDEX_TO_CHAR)}
dict_out = {v:k for k, v in dict_in.items()}

MAX_LEN       = 20

def convert_to_torch(text):
    output = torch.zeros((len(text), MAX_LEN), dtype=int)
    for i in range(len(text)):
        for j, w in enumerate(text[i]):
            if j >= MAX_LEN:
                break
            output[i, j] = dict_in.get(w, dict_in['none'])

    return output

X_train= convert_to_torch(train_text)
Y_train= convert_to_torch(train_label)
X_test = convert_to_torch(test_text)
Y_test = convert_to_torch(test_label)

In [14]:
class RNN_Network(torch.nn.Module):
    def __init__(self):
        super(RNN_Network, self).__init__()
        self.embeddings = torch.nn.Embedding(len(dict_in), 28)
        self.rnn = torch.nn.RNN(28, 256, batch_first=True)
        self.linear = torch.nn.Linear(256, 28)

    def forward(self, sentences, state=None):
        embds = self.embeddings(sentences)
        out, new_state = self.rnn(embds, state)
        result = self.linear(out)
        return result, new_state

In [15]:
model = RNN_Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
epochs = 20

In [16]:
for epoch in range(epochs):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    test_loss = 0
    test_passed = 0

    for i in range(int(len(X_train) / 100)):
        X_batch = X_train[i * 100:(i + 1) * 100]
        Y_batch = Y_train[i * 100:(i + 1) * 100].flatten()
        
        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(dict_in))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        train_passed += 1

    with torch.no_grad():
        model.eval()
        answers, _ = model.forward(X_test)
        answers = answers.view(-1, len(dict_in))
        loss = criterion(answers, Y_test.flatten())
        test_loss += loss.item()
        test_passed += 1

    print(f"Epoch {epoch + 1}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}, Test loss: {test_loss / test_passed:.6f}")

Epoch 1. Time: 2.104, Train loss: 1.691, Test loss: 1.308471
Epoch 2. Time: 1.717, Train loss: 1.251, Test loss: 1.205643
Epoch 3. Time: 1.742, Train loss: 1.160, Test loss: 1.115506
Epoch 4. Time: 1.743, Train loss: 1.040, Test loss: 0.970439
Epoch 5. Time: 1.751, Train loss: 0.879, Test loss: 0.828048
Epoch 6. Time: 1.709, Train loss: 0.729, Test loss: 0.701996
Epoch 7. Time: 1.689, Train loss: 0.627, Test loss: 0.614614
Epoch 8. Time: 1.731, Train loss: 0.555, Test loss: 0.554984
Epoch 9. Time: 1.724, Train loss: 0.484, Test loss: 0.506925
Epoch 10. Time: 1.814, Train loss: 0.424, Test loss: 0.463546
Epoch 11. Time: 1.806, Train loss: 0.380, Test loss: 0.429090
Epoch 12. Time: 1.824, Train loss: 0.350, Test loss: 0.417009
Epoch 13. Time: 1.749, Train loss: 0.330, Test loss: 0.400723
Epoch 14. Time: 1.742, Train loss: 0.311, Test loss: 0.381552
Epoch 15. Time: 1.725, Train loss: 0.289, Test loss: 0.355932
Epoch 16. Time: 1.723, Train loss: 0.260, Test loss: 0.354104
Epoch 17. Time: 1

###Задание 2.

На основе лекционного ноутбука.

а) построить RNN-ячейку на основе полносвязных слоев

б) применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [17]:
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}
MAX_LEN = 50
n_classes = len(INDEX_TO_CHAR)
X = convert_to_torch(df['text_in'].tolist())

In [18]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 812, batch_first=True)
        self.linear = torch.nn.Linear(812, len(INDEX_TO_CHAR))
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [19]:
def generate_sentence(txt_list):

    sentence =  [c for c in txt_list]
    x = torch.zeros((1, len(sentence)), dtype=int).to(dev)
    
    for j,w in enumerate(sentence):
        if j>=MAX_LEN:
            break
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
        
    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1, :], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0).unsqueeze(1)], axis=1)
        ww = INDEX_TO_CHAR[w]
        if ww == 'none':
            break

        sentence.append(ww)

    return ''.join(sentence)

In [20]:
model     = Network().to(dev)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.0001)

phrase = 'youre a great mom you were always '

In [21]:
for ep in range(200):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100] 
        X_batch = batch[:, :-1].to(dev)
        Y_batch = batch[:, 1:].flatten().to(dev) 

        optimizer.zero_grad()
        answers = model(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch).to(dev)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1     

    if ep%5 == 0: 
        print("\nEpoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        s = generate_sentence(phrase)
        print(s)
    else:
        print(f"\rEpoch {ep}, loss: {train_loss / train_passed:.3f}", end='') 


Epoch 0. Time: 1.500, Train loss: 2.104
youre a great mom you were always to to to to to to to to to to to to to to to to to
Epoch 4, loss: 1.558
Epoch 5. Time: 1.243, Train loss: 1.524
youre a great mom you were always are the the the the the the the the the the the th
Epoch 9, loss: 1.424
Epoch 10. Time: 1.257, Train loss: 1.402
youre a great mom you were always the some his the some hing the some bert in the so
Epoch 14, loss: 1.328
Epoch 15. Time: 1.268, Train loss: 1.312
youre a great mom you were always the bart the some his the bart the some his the ba
Epoch 19, loss: 1.253
Epoch 20. Time: 1.278, Train loss: 1.240
youre a great mom you were always the bart was the berter the some hive to see the b
Epoch 24, loss: 1.191
Epoch 25. Time: 1.282, Train loss: 1.181
youre a great mom you were always the best simeshing the serest is the bart was the 
Epoch 29, loss: 1.141
Epoch 30. Time: 1.282, Train loss: 1.132
youre a great mom you were always the best simpson and the serest is a sto